In [ ]:
import pandas as pd
import numpy as np
import json
from collections import Counter
import copy

In [3]:
# Datasets and list of synonyms path
commit_sharing = "./20231012_230826_commit_sharings.json"
pull_request_sharing = "./20231012_233628_pr_sharings.json"
issue_sharing = "./20231012_235128_issue_sharings.json"
file_sharings = "./20231012_234250_file_sharings.json"
discussion_sharings = "./20231012_235320_discussion_sharings.json"

path_list_synonym = "./synonyms_for_normalization.txt"

In [ ]:
jsons_list = [commit_sharing,pull_request_sharing,issue_sharing,file_sharings,discussion_sharings]
jsons_file = ['commit_sharing','pull_request_sharing','issue_sharing','file_sharings','discussion_sharings']

In [ ]:
# 0 = commit_sharing, 1 = pull_request_sharing, 2 = issue_sharing, 3 = file_sharings, 4 = discussion_sharings
fileNumber = 4

with open(jsons_list[fileNumber], 'r') as f:
    data1 = json.load(f)

# Normalisation

In [ ]:
# Pre-processing synonym_list file
with open(path_list_synonym, 'r') as arquivo:
    # dict to store the synonyms
    synonyms = {}
    for linha in arquivo:
        chave, valor = linha.strip().split('=')

        synonyms[chave.strip()] = valor.strip().split(',')


In [ ]:

for elementos in data1["Sources"]:
  if elementos["RepoLanguage"] == None :
      elementos["RepoLanguage"] = "none"
  else:
    for linguagem, alias in synonyms.items():
      if elementos["RepoLanguage"] in alias:
        elementos["RepoLanguage"] = linguagem



for elementos in data1["Sources"]:
  for elemento in elementos["ChatgptSharing"]:
    if "Conversations" in elemento and elemento["Conversations"]:
      for conversation in elemento["Conversations"]:
        if "ListOfCode" in conversation and conversation["ListOfCode"]:
          for snipetType in conversation["ListOfCode"]:
            if snipetType["Type"] == None :
              snipetType["Type"] = "none"
            else:
              for linguagem, alias in synonyms.items():
                if  snipetType["Type"] in alias:
                  snipetType["Type"] = linguagem


In [ ]:

# write processed json
with open("./normalised/"+jsons_file[j]+'.json', 'w') as arquivo:
  json.dump(data1, arquivo)

# Data Extraction

In [ ]:
repoLang = []
for elementos in data1["Sources"]:
  repoLang.append(elementos["RepoLanguage"])


In [ ]:
counting = Counter(repoLang)

countingDict = dict(counting)

In [ ]:
df0 = pd.DataFrame(list(countingDict.items()), columns=['Language', 'Count'])

In [ ]:
repoByLanguage = {}

for key in counting.keys():
  repoByLanguage.setdefault(key, [])
  for elements in data1["Sources"]:
    if elements["RepoLanguage"] == key:
      repoByLanguage[key].append(elements["RepoName"])


In [ ]:
df1 = pd.DataFrame(list(repoByLanguage.items()), columns=['Language', 'Repos'])

In [ ]:
repoByLanguageCount = copy.deepcopy(repoByLanguage)

for key, alias in repoByLanguage.items():
    repoByLanguage[key] = list(set(alias))
    repoByLanguageCount[key] = len(list(set(alias)))

In [ ]:
df2 = pd.DataFrame(list(repoByLanguage.items()), columns=['Language', 'UniqueRepos'])
df3 = pd.DataFrame(list(repoByLanguageCount.items()), columns=['Language', 'UniqueReposCount'])

## CountChat

In [ ]:
chatByLanguage = {}
for elements in data1["Sources"]:
  chatByLanguage.setdefault(elements["RepoLanguage"], [])
  for element in elements["ChatgptSharing"]:
    chatByLanguage[elements["RepoLanguage"]].append(element["URL"])



In [ ]:
df4 = pd.DataFrame(list(chatByLanguage.items()), columns=['Language', 'Chat'])

In [ ]:
chatByLanguageCount = copy.deepcopy(chatByLanguage)
for key, alias in chatByLanguage.items():
    chatByLanguage[key] = list(set(alias))
    chatByLanguageCount[key] = len(list(set(alias)))

In [ ]:
chatByLanguage
df5 = pd.DataFrame(list(chatByLanguage.items()), columns=['Language', 'UniqueChat'])
df6 = pd.DataFrame(list(chatByLanguageCount.items()), columns=['Language', 'UniqueChatCount'])

## Code Snipets

In [ ]:
snipetsByLanguage = {}
for elements in data1["Sources"]:
  snipetsByLanguage.setdefault(elements["RepoLanguage"], [])
  for element in elements["ChatgptSharing"]:
    if "Conversations" in element and element["Conversations"]:
      for conversation in element["Conversations"]:
        if "ListOfCode" in conversation and conversation["ListOfCode"]:
          for snipetType in conversation["ListOfCode"]:
            snipetsByLanguage[elements["RepoLanguage"]].append(snipetType["Type"])



In [ ]:
snipetsByLanguage
snipetsByLanguagePercentage = copy.deepcopy(snipetsByLanguage)
snipetsByLanguageTotal = copy.deepcopy(snipetsByLanguage)

In [ ]:

for key, alias in snipetsByLanguage.items():
    total_elements_on_lista = len(alias)

    count_elements = Counter(alias)

    snipetsByLanguagePercentage[key] = {element: f'{(ocurrency / total_elements_on_lista) * 100:.3f}' for element, ocurrency in count_elements.items()}
    snipetsByLanguage[key] = dict(Counter(snipetsByLanguage[key]))
    snipetsByLanguageTotal[key] = (sum(Counter(snipetsByLanguage[key]).values()))

In [ ]:
df7 = pd.DataFrame(list(snipetsByLanguage.items()), columns=['Language', 'SnippetLang'])

df8 = pd.DataFrame(list(snipetsByLanguageTotal.items()), columns=['Language', 'SnippetLangCount'])

df9 = pd.DataFrame(list(snipetsByLanguagePercentage.items()), columns=['Language', 'SnippetLangPercentage'])

df_concat = pd.merge(df0, df1, on='Language').merge(df2, on='Language').merge(df3, on='Language').merge(df4, on='Language')
df_concat = df_concat.merge(df5, on='Language').merge(df6, on='Language').merge(df7, on='Language').merge(df8, on='Language').merge(df9, on='Language')
df_concat.to_csv(jsons_file[j]+'_results_table.csv', index=False)

## Verify Snipets by language

In [ ]:

rows = []
for columns, values in snipetsByLanguage.items():
    row = {'language': columns}
    row.update(values)
    rows.append(row)

snipetsByLanguageDF = pd.DataFrame(rows)
snipetsByLanguageDF = snipetsByLanguageDF.T


snipetsByLanguageDF = snipetsByLanguageDF.fillna(0)
snipetsByLanguageDF.to_csv(jsons_file[j]+'_snipetsByLanguageDF.csv')

## Snipets By Language in Percentage

In [ ]:
rows = []
for columns, values in snipetsByLanguagePercentage.items():
    row = {'language': columns}
    row.update(values)
    rows.append(row)

snipetsByLanguagePercentageDF = pd.DataFrame(rows)
snipetsByLanguagePercentageDF = snipetsByLanguagePercentageDF.T

snipetsByLanguagePercentageDF = snipetsByLanguagePercentageDF.fillna(0)
snipetsByLanguagePercentageDF.to_csv(jsons_file[j]+'_snipetsByLanguagePercentageDF.csv')